# GeoSparse vs ReLU Baselines (psann==0.12.2)

**Goal:** Compare GeoSparse against dense ReLU baselines on diverse regression datasets,
with **parameter-matched** models and **wall-clock timing**.

**Fairness contract:**
- Same optimizer family + schedule (Adam, fixed LR).
- Same batch size and **target optimizer steps** across datasets.
- Same preprocessing policy (StandardScaler on X; optional y-scaling).
- Parameter counts matched within 1%.

This notebook is **Colab-ready** and uses the PyPI distribution only.


## Reproduce in Colab

- Runtime: **GPU** (recommended) or **CPU**.
- Python: Colab default (3.10+).
- Expected runtime (quick mode): ~5-10 minutes on a free GPU.
- Expected runtime (full mode, 3 seeds): ~20-40 minutes.

Toggle `QUICK_MODE` in the reproducibility section for a fast smoke run.


In [ ]:
# Install minimal dependencies from PyPI only.
%pip install -q psann==0.12.2 scikit-learn pandas matplotlib


In [ ]:
import platform
import psann
import torch
import numpy as np
import sklearn

print("python:", platform.python_version())
print("psann:", psann.__version__)
print("psann file:", psann.__file__)
print("torch:", torch.__version__)
print("numpy:", np.__version__)
print("sklearn:", sklearn.__version__)

if torch.cuda.is_available():
    print("cuda:", torch.version.cuda)
    print("gpu:", torch.cuda.get_device_name(0))
else:
    print("cuda: not available")


We start by defining the dataset suite and a loader with a shared signature.
Each dataset returns `(X, y, task_type, feature_names, target_name)` so the
training harness can be uniform.


## Dataset suite (frozen list)

**Synthetic (regression-only):**
- `syn_sparse_linear`: high-dimensional sparse linear regression (n_features=200, n_informative=10).
- `syn_friedman1`: nonlinear interactions via Friedman-1.
- `syn_piecewise_sine`: periodic + regime-switch target with noise.

**Real (regression-only):**
- `real_california_housing`: California Housing (fetch).
- `real_diabetes`: Diabetes dataset (load).
- `real_linnerud`: Linnerud dataset (load; uses single target column).


In [ ]:
import numpy as np

from sklearn.datasets import (
    make_regression,
    make_friedman1,
    fetch_california_housing,
    load_diabetes,
    load_linnerud,
)

DATASET_NAMES = [
    'syn_sparse_linear',
    'syn_friedman1',
    'syn_piecewise_sine',
    'real_california_housing',
    'real_diabetes',
    'real_linnerud',
]

def load_dataset(name, seed=1337):
    # Returns: X, y, task_type, feature_names, target_name
    if name == 'syn_sparse_linear':
        X, y = make_regression(
            n_samples=5000,
            n_features=200,
            n_informative=10,
            noise=5.0,
            bias=10.0,
            random_state=seed,
        )
        feature_names = [f'x{i}' for i in range(X.shape[1])]
        target_name = 'y'
    elif name == 'syn_friedman1':
        X, y = make_friedman1(
            n_samples=5000,
            n_features=20,
            noise=0.5,
            random_state=seed,
        )
        feature_names = [f'x{i}' for i in range(X.shape[1])]
        target_name = 'y'
    elif name == 'syn_piecewise_sine':
        rng = np.random.default_rng(seed)
        n_samples = 5000
        n_features = 10
        X = rng.normal(size=(n_samples, n_features))
        t = X[:, 0]
        # Regime switch at t=0 with different frequencies and offsets.
        y = np.where(
            t < 0,
            0.5 * np.sin(3.0 * t) + 0.2 * t,
            1.0 * np.sin(6.0 * t + 0.5) - 0.1 * t + 0.5,
        )
        y = y + 0.1 * rng.normal(size=n_samples)
        feature_names = [f'x{i}' for i in range(X.shape[1])]
        target_name = 'y'
    elif name == 'real_california_housing':
        data = fetch_california_housing()
        X, y = data.data, data.target
        feature_names = list(data.feature_names)
        target_name = 'MedHouseVal'
    elif name == 'real_diabetes':
        data = load_diabetes()
        X, y = data.data, data.target
        feature_names = list(data.feature_names)
        target_name = 'disease_progression'
    elif name == 'real_linnerud':
        data = load_linnerud()
        X = data.data
        # Use a single target column for regression comparability.
        target_index = 0
        y = data.target[:, target_index]
        feature_names = list(data.feature_names)
        target_name = data.target_names[target_index]
    else:
        raise ValueError(f'Unknown dataset: {name}')

    X = np.asarray(X, dtype=np.float32)
    y = np.asarray(y, dtype=np.float32)
    task_type = 'regression'
    return X, y, task_type, feature_names, target_name

print('Datasets:', DATASET_NAMES)


We standardize inputs and optionally scale targets. Metrics are always reported
on **unscaled** targets so results remain interpretable.


## Preprocessing & splits

**Policy:**
- Train/val/test split uses a fixed seed for reproducibility.
- `StandardScaler` is fit on **train** data only, then applied to val/test.
- Optional target scaling for regression is supported; metrics should be
  computed on **unscaled** targets via inverse-transform.

**Shape conventions:**
- `X`: `(n_samples, n_features)`
- `y`: `(n_samples,)` for regression


In [ ]:
from dataclasses import dataclass
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

SPLIT_SEED = 1337
TEST_SIZE = 0.2
VAL_SIZE = 0.1
SCALE_Y = False

@dataclass
class DatasetSplit:
    X_train: np.ndarray
    y_train: np.ndarray
    X_val: np.ndarray
    y_val: np.ndarray
    X_test: np.ndarray
    y_test: np.ndarray
    x_scaler: StandardScaler
    y_scaler: StandardScaler | None

def split_and_scale(X, y, seed=SPLIT_SEED, test_size=TEST_SIZE, val_size=VAL_SIZE, scale_y=SCALE_Y):
    # First split train vs (val+test).
    X_train, X_tmp, y_train, y_tmp = train_test_split(
        X, y, test_size=test_size + val_size, random_state=seed
    )
    # Split val vs test from the temp set.
    val_frac = val_size / (test_size + val_size)
    X_val, X_test, y_val, y_test = train_test_split(
        X_tmp, y_tmp, test_size=1.0 - val_frac, random_state=seed
    )

    x_scaler = StandardScaler().fit(X_train)
    X_train_s = x_scaler.transform(X_train).astype(np.float32, copy=False)
    X_val_s = x_scaler.transform(X_val).astype(np.float32, copy=False)
    X_test_s = x_scaler.transform(X_test).astype(np.float32, copy=False)

    y_scaler = None
    if scale_y:
        y_scaler = StandardScaler().fit(y_train.reshape(-1, 1))
        y_train_s = y_scaler.transform(y_train.reshape(-1, 1)).ravel()
        y_val_s = y_scaler.transform(y_val.reshape(-1, 1)).ravel()
        y_test_s = y_scaler.transform(y_test.reshape(-1, 1)).ravel()
    else:
        y_train_s, y_val_s, y_test_s = y_train, y_val, y_test

    y_train_s = y_train_s.astype(np.float32, copy=False)
    y_val_s = y_val_s.astype(np.float32, copy=False)
    y_test_s = y_test_s.astype(np.float32, copy=False)

    return DatasetSplit(
        X_train=X_train_s, y_train=y_train_s,
        X_val=X_val_s, y_val=y_val_s,
        X_test=X_test_s, y_test=y_test_s,
        x_scaler=x_scaler, y_scaler=y_scaler,
    )


## Models under test

We compare **GeoSparse** against a **dense ReLU baseline** using the public
`psann` sklearn-style estimators.

- **GeoSparse:** `GeoSparseRegressor` (activation_type=`"psann"` by default).
- **Dense ReLU:** `PSANNRegressor` with `activation_type="relu"`.

**Shape note:** `GeoSparseRegressor` requires a 2D shape `(H, W)` such that
`H * W == n_features`. For tabular datasets we use `(1, n_features)` by default.


In [ ]:
from psann import GeoSparseRegressor, PSANNRegressor

DEFAULT_TRAIN_CFG = dict(
    epochs=100,
    batch_size=256,
    lr=1e-3,
    optimizer='adam',
    weight_decay=0.0,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    random_state=1337,
)

def resolve_geo_shape(input_dim, shape=None):
    if shape is not None:
        return tuple(shape)
    return (1, int(input_dim))

def build_geosparse_regressor(input_dim, *, shape=None, activation_type='psann', **overrides):
    cfg = dict(DEFAULT_TRAIN_CFG)
    cfg.update(overrides)
    return GeoSparseRegressor(
        hidden_layers=4,
        activation_type=activation_type,
        shape=resolve_geo_shape(input_dim, shape),
        k=8,
        compute_mode='gather',
        **cfg,
    )

def build_dense_relu_regressor(hidden_layers=2, hidden_units=64, **overrides):
    cfg = dict(DEFAULT_TRAIN_CFG)
    cfg.update(overrides)
    return PSANNRegressor(
        hidden_layers=hidden_layers,
        hidden_units=hidden_units,
        activation_type='relu',
        **cfg,
    )


## Parameter matching (GeoSparse vs Dense ReLU)

We match **total trainable parameters** by counting parameters of the
underlying PyTorch modules (including PSANN sine parameters).
Dense ReLU width is chosen to minimize absolute mismatch within a tolerance.


In [ ]:
from psann import count_params
from psann.nn import PSANNNet
from psann.nn_geo_sparse import GeoSparseNet

PARAM_TOL = 0.01  # <= 1% relative mismatch

def count_geosparse_params(input_dim, output_dim, *, depth=4, k=8, shape=None, activation_type='psann'):
    shape = resolve_geo_shape(input_dim, shape)
    model = GeoSparseNet(
        int(input_dim),
        int(output_dim),
        shape=shape,
        depth=int(depth),
        k=int(k),
        activation_type=activation_type,
        norm='rms',
        drop_path_max=0.0,
        residual_alpha_init=0.0,
        bias=True,
        compute_mode='gather',
        seed=1337,
    )
    return count_params(model, trainable_only=True)

def count_dense_params(input_dim, output_dim, *, depth=2, hidden_units=64, activation_type='relu'):
    model = PSANNNet(
        int(input_dim),
        int(output_dim),
        hidden_layers=int(depth),
        hidden_units=int(hidden_units),
        activation_type=activation_type,
    )
    return count_params(model, trainable_only=True)

def match_dense_width_to_geosparse(
    input_dim,
    output_dim=1,
    *,
    geo_depth=4,
    geo_k=8,
    geo_shape=None,
    dense_depth=2,
    width_candidates=None,
    tol=PARAM_TOL,
):
    target = count_geosparse_params(
        input_dim, output_dim, depth=geo_depth, k=geo_k, shape=geo_shape
    )
    if width_candidates is None:
        width_candidates = list(range(8, 2049, 8))
    best_width = None
    best_params = None
    best_mismatch = None
    for width in width_candidates:
        params = count_dense_params(
            input_dim, output_dim, depth=dense_depth, hidden_units=width, activation_type='relu'
        )
        mismatch = abs(params - target)
        if best_mismatch is None or mismatch < best_mismatch:
            best_mismatch = mismatch
            best_width = width
            best_params = params
    rel_mismatch = best_mismatch / max(1, target)
    if rel_mismatch > tol:
        print(f'WARN: best rel mismatch {rel_mismatch:.3f} exceeds tol={tol:.3f}')
    return {
        'target_params': target,
        'dense_width': best_width,
        'dense_params': best_params,
        'rel_mismatch': rel_mismatch,
    }

# Example sanity check for a 20-feature dataset
example = match_dense_width_to_geosparse(input_dim=20, output_dim=1)
example


We match training budgets by optimizer steps instead of epochs to keep
comparisons consistent across datasets with different sizes.


## Training loop + timing

We time the full `fit(...)` call with CUDA synchronization when available.
A short *warmup* run is executed on a throwaway model to avoid timing
first-iteration kernel compilation.

**Budget policy:** target a fixed **number of optimizer steps** per run
by adjusting epochs based on the dataset size and batch size.


In [ ]:
import math
import time
import types

TARGET_STEPS = 400
PROGRESS_EVERY_STEPS = 100
WARMUP_SAMPLES = 512
WARMUP_EPOCHS = 1

def _maybe_cuda_sync():
    if torch.cuda.is_available():
        torch.cuda.synchronize()

def _subsample(X, y, n_samples, seed=1337):
    if n_samples is None or len(X) <= n_samples:
        return X, y
    rng = np.random.default_rng(seed)
    idx = rng.choice(len(X), size=int(n_samples), replace=False)
    return X[idx], y[idx]

def _attach_progress(model, steps_per_epoch, progress_every_steps=PROGRESS_EVERY_STEPS):
    state = {'next_report': progress_every_steps}
    def _epoch_callback(self, epoch, train_loss, val_loss, improved, patience_left):
        steps_done = (epoch + 1) * steps_per_epoch
        if steps_done >= state['next_report']:
            msg = f"[step ~{steps_done}] train_loss={train_loss:.4f}"
            if val_loss is not None:
                msg += f" val_loss={val_loss:.4f}"
            print(msg)
            state['next_report'] += progress_every_steps
    model.epoch_callback = types.MethodType(_epoch_callback, model)
    return model

def _compute_epochs(n_train, batch_size, target_steps=TARGET_STEPS):
    steps_per_epoch = int(math.ceil(n_train / batch_size))
    epochs = int(math.ceil(target_steps / steps_per_epoch))
    return max(1, epochs), steps_per_epoch

def warmup_fit(model_factory, X_train, y_train, X_val, y_val):
    if not torch.cuda.is_available():
        return
    X_w, y_w = _subsample(X_train, y_train, WARMUP_SAMPLES, seed=123)
    model = model_factory(epochs=WARMUP_EPOCHS)
    _maybe_cuda_sync()
    model.fit(X_w, y_w, validation_data=(X_val, y_val), verbose=0)
    _maybe_cuda_sync()

def fit_with_timing(model_factory, X_train, y_train, X_val, y_val, *, batch_size):
    epochs, steps_per_epoch = _compute_epochs(len(X_train), batch_size)
    model = model_factory(epochs=epochs)
    _attach_progress(model, steps_per_epoch)

    _maybe_cuda_sync()
    start = time.perf_counter()
    model.fit(X_train, y_train, validation_data=(X_val, y_val), verbose=0)
    _maybe_cuda_sync()
    elapsed = time.perf_counter() - start

    total_steps = steps_per_epoch * epochs
    samples_seen = len(X_train) * epochs
    steps_per_sec = total_steps / max(elapsed, 1e-9)
    samples_per_sec = samples_seen / max(elapsed, 1e-9)

    return {
        'model': model,
        'epochs': epochs,
        'steps_per_epoch': steps_per_epoch,
        'total_steps': total_steps,
        'train_time_s': elapsed,
        'steps_per_sec': steps_per_sec,
        'samples_per_sec': samples_per_sec,
    }


## Evaluation & reporting

We compute regression metrics on **unscaled** targets (if `SCALE_Y=True`,
we inverse-transform `y` and predictions for reporting).


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def compute_regression_metrics(y_true, y_pred):
    return {
        'mse': mean_squared_error(y_true, y_pred),
        'mae': mean_absolute_error(y_true, y_pred),
        'r2': r2_score(y_true, y_pred),
    }

def _unscale_y(y, y_scaler):
    if y_scaler is None:
        return y
    return y_scaler.inverse_transform(y.reshape(-1, 1)).ravel()

def evaluate_regression(model, split):
    y_true = _unscale_y(split.y_test, split.y_scaler)
    y_pred = model.predict(split.X_test)
    y_pred = _unscale_y(np.asarray(y_pred, dtype=np.float32), split.y_scaler)
    return compute_regression_metrics(y_true, y_pred)

def build_results_table(rows):
    df = pd.DataFrame(rows)
    if df.empty:
        return df, df
    summary = (
        df.groupby(['dataset', 'model'])
        .agg(
            mse_mean=('mse', 'mean'),
            mse_std=('mse', 'std'),
            mae_mean=('mae', 'mean'),
            mae_std=('mae', 'std'),
            r2_mean=('r2', 'mean'),
            r2_std=('r2', 'std'),
            train_time_mean=('train_time_s', 'mean'),
            train_time_std=('train_time_s', 'std'),
            params_mean=('params', 'mean'),
        )
        .reset_index()
    )
    return df, summary

def plot_metric(df, metric, title):
    if df.empty:
        print('No results to plot yet.')
        return
    ax = df.pivot_table(index='dataset', columns='model', values=metric).plot(kind='bar')
    ax.set_title(title)
    ax.set_ylabel(metric)
    plt.show()

def plot_train_time(df):
    if df.empty:
        print('No results to plot yet.')
        return
    ax = df.pivot_table(index='dataset', columns='model', values='train_time_s').plot(kind='bar')
    ax.set_title('Train wall time (s)')
    ax.set_ylabel('seconds')
    plt.show()

# Placeholder for experiment results
results = []

# Example usage after running experiments:
# df, summary = build_results_table(results)
# display(df.head())
# display(summary)
# plot_metric(summary, 'mse_mean', 'MSE by dataset/model')
# plot_train_time(summary)


## Reproducibility & stability checks

We run multiple seeds and apply simple sanity checks so results are trustworthy.
A `QUICK_MODE` option reduces runtime for smoke tests.


In [ ]:
import random

SEEDS = [0, 1, 2]
QUICK_MODE = True

if QUICK_MODE:
    SEEDS = [0]
    TARGET_STEPS = 150
    print('QUICK_MODE enabled: seeds=', SEEDS, 'steps=', TARGET_STEPS)

def seed_all(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def sanity_check_param_matching(input_dim, output_dim=1):
    match = match_dense_width_to_geosparse(input_dim=input_dim, output_dim=output_dim)
    rel = match['rel_mismatch']
    if rel > PARAM_TOL:
        print('WARN: param mismatch above tolerance:', match)
    return match

def sanity_check_results(rows):
    if not rows:
        print('No results yet; run experiments first.')
        return
    for row in rows:
        if row.get('train_time_s', 0.0) <= 0:
            print('WARN: non-positive train_time_s:', row)
        if row.get('params') is None:
            print('WARN: missing params:', row)

# Example: sanity check for each dataset dimension
for name in DATASET_NAMES:
    X, y, task, feats, target = load_dataset(name)
    sanity_check_param_matching(input_dim=X.shape[1], output_dim=1)

# After running experiments: sanity_check_results(results)


## Conclusions (fill after running)

- Summary of GeoSparse vs ReLU performance across datasets.
- Where GeoSparse helps or hurts.
- Suggested next tests (more datasets, larger budgets, classification add-on).
